In [ ]:
import pandas as pd
# import matplotlib.pyplot as plt
# import matplotlib.text as txt
# import matplotlib as mpl
# import numpy as np
# import seaborn as sb
# from pathlib import Path
# import os

import sys
from time import sleep

def progress_bar(progress: int = 0, total: int = 100, dim: int = 100, char: str = '█', file: str = "") -> None:
    percent = 100 * float(progress / float(total))
    alg_percent = dim * (progress / float(total))
    bar = char * int(alg_percent) + '-' * (dim - int(alg_percent))
    print(f"\r|{bar}| {percent:.4f}% - {progress}/{total} - {file}", end = "\r")

lineNumber = 0

# Qui ottengo la lista dei file in /shares/../Mar22
f = open("ls_Mar22.txt", "r")

lines = f.readlines()
f.close()
# os.listdir(path)
path_db = "/shares/CIBIO-Storage/CM/scratch/databases/MetaRefSGB/pwd/Mar22/"

f = open("simple_statistical_results/sgb_statistics_table.txt","w")
f.write(
    "SGB\t" +
    "Number of genomes\t" +
    "Average distance\t" +
    "Max distance\t" +
    "Min distance\t" +
    "Max average distance of rows\t" +
    "Min average distance of rows\t" +
    "Number of outliers\n"
)

# Qui per ogni file
# lines = lines[:500]
# n_simbols = 70 / len(lines)
for lineNumber, file_in_lines in enumerate(lines):
    
    # ------ Progress Bar (Total) --------
#     sys.stdout.write('\r')
#     sys.stdout.write("[{:{}}] {:.4f}% - {actual}/{total} - {file}   ".format("="*round(lineNumber*n_simbols), 70, (100/(len(lines))*lineNumber), actual = lineNumber, total = len(lines), file = file_in_lines[:-1]))
#     sys.stdout.flush()
    progress_bar(lineNumber + 1, len(lines), 70, char = '=', file = file_in_lines[:-1])
    
    # Percorso del file da aprire
    file = path_db + file_in_lines[:-1]
    
    # Settaggio del DataFrame del file
    df = pd.read_csv(file, sep = "\t")
    df = df.rename(columns = {"Unnamed: 0": ""})
    df = df.set_index("")
    
    if (len(df.index) > 1):
        # Settaggio delle variabili necessarie
        avgg = 0.0                  # Average distance
        maxx = 0.0                  # Max distance
        minn = float('inf')         # Min distance
        avgg_max = 0.0              # Max average distance of rows
        avgg_min = float('inf')     # Min average distance of rows
        counter_avgg_max = 0        # Number of outliers

        # Lettura del DataFrame del file
        for i in df.index:                 # Per ogni riga
            avgg_rel = 0.0                     # Media per riga
            
            for j in df.index:                 # Per ogni colonna
                value = df.loc[i][j]               # Ignora diagonale
                if i != j:
                    avgg += value                  # avg parziale
                    avgg_rel += value
                    if value > maxx:               # max
                        maxx = value
                    if value < minn:               # min
                        minn = value
            
            avgg_rel /= len(df.index) - 1
            
            if avgg_rel > avgg_max:            # max avg per riga
                avgg_max = avgg_rel
            if avgg_rel < avgg_min:            # min avg per riga
                avgg_min = avgg_rel
            if avgg_rel > 0.05:                # n° outliers
                counter_avgg_max += 1

        avgg /= len(df.index)**2 - len(df.index)

        # Outputs
        f.write(
            file_in_lines[:-5] + "\t"        # SGB
            + str(len(df.index)) + "\t"      # Number of genomes
            + str(float(avgg)) + "\t"        # Average distance
            + str(maxx) + "\t"               # Max distance
            + str(minn) + "\t"               # Min distance
            + str(avgg_max) + "\t"           # Max average distance of rows
            + str(avgg_min) + "\t"           # Min average distance of rows
            + str(counter_avgg_max) + "\n"   # Number of outliers
        )
    # End if

    # ------ Progress Bar (Total) --------
#     sys.stdout.write('\r')
#     sys.stdout.write("[{:{}}] {:.4f}% - {actual}/{total} - {file}   ".format("="*round(lineNumber*n_simbols), 70, (100/(len(lines))*lineNumber), actual = lineNumber, total = len(lines), file = file_in_lines[:-1]))
#     sys.stdout.flush()
#     progress_bar(lineNumber + 1, len(lines), 70, char = '=')
# End for
    
# Chiusura del file per i massimi delle medie
f.close()
print("\n\nEnd process!!")

In [50]:
import pandas as pd

outdf = pd.read_csv("simple_statistical_results/sgb_statistics_table.txt", sep = "\t")
outdf.sort_values(by=['Max distance'])
# outdf[outdf["Number of outliers"] > 0]

,SGB,Number of genomes,Average distance,Max distance,Min distance,Max average distance of rows,Min average distance of rows,Number of outliers
3527,15949,2,0.000000,0.0,0.000000,0.000000,0.000000,0
298,10266,2,0.000000,0.0,0.000000,0.000000,0.000000,0
2058,13666,2,0.000000,0.0,0.000000,0.000000,0.000000,0
3967,16751,2,0.000000,0.0,0.000000,0.000000,0.000000,0
2052,13658,2,0.000000,0.0,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...
4287,17244,3686,0.028477,1.0,0.000021,0.076319,0.019459,12
4281,17237,1555,0.030503,1.0,0.000007,0.087663,0.021230,23
4284,17241,446,0.034844,1.0,0.000175,0.069299,0.024715,16
4092,16955,623,0.014653,1.0,0.000000,0.047653,0.009171,0
